In [2]:
from platform import python_version
print(python_version())

3.9.17


In [3]:
import pandas as pd
import numpy as np
import math
import pickle # get the ML model from other notebook
from matplotlib import pyplot as plt # scatter plot
import matplotlib.lines as mlines # plot

In [4]:
# Set the plotting DPI settings to be a bit higher.
plt.rcParams['figure.figsize'] = [7.0, 4.5]
plt.rcParams['figure.dpi'] = 150

# get_x_y_raw_data

In [18]:
def getYPriceDataNearDate(ticker, date, modifier, dailySharePrices):
    '''
    Return just the y price and volume.
    Take the first day price/volume of the list of days,
    that fall in the window of accepted days.
    'modifier' just modifies the date to look between.
    Returns a list.
    '''
    windowDays=5
    rows = dailySharePrices[
        (dailySharePrices["Date"].between(pd.to_datetime(date)
                                          + pd.Timedelta(days=modifier),
                                          pd.to_datetime(date)
                                          + pd.Timedelta(days=windowDays
                                                         +modifier)
                                         )
        ) & (dailySharePrices["Ticker"]==ticker)]
    
    if rows.empty:
        return [ticker, np.nan,\
                np.datetime64('NaT'),\
                np.nan]
    else:
        return [ticker, rows.iloc[0]["Open"],\
                rows.iloc[0]["Date"],\
                rows.iloc[0]["Volume"]*rows.iloc[0]["Open"]]

In [19]:
def getYPricesReportDateAndTargetDate(x, d, modifier=365):
    '''
    Takes in all fundamental data X, all stock prices over time y,
    and modifier (days), and returns the stock price info for the
    data report date, as well as the stock price one year from that date
    (if modifier is left as modifier=365)
    '''
    # Preallocation list of list of 2 
    # [(price at date) (price at date + modifier)]
    y = [[None]*8 for i in range(len(x))] 
    
    whichDateCol='Publish Date'# or 'Report Date', 
    # is the performance date from->to. Want this to be publish date.
    
    # Because of time lag between report date
    # (which can't be actioned on) and publish date
    # (data we can trade with)
    
    # In the end decided this instead of iterating through index.
    # Iterate through a range rather than index, as X might not have
    # monotonic increasing index 1, 2, 3, etc.
    i=0
    for index in range(len(x)):
        y[i]=(getYPriceDataNearDate(x['Ticker'].iloc[index], 
                                    x[whichDateCol].iloc[index],0,d)
              +getYPriceDataNearDate(x['Ticker'].iloc[index], 
                                     x[whichDateCol].iloc[index], 
                                     modifier, d))
        i=i+1
        
    return y

In [12]:
def getYRawData2024(my_path = 'C:/Users/damon/OneDrive/BYO_Investing_AI/2024/Stock_Data/SimFin2024/'):
    d=pd.read_csv(my_path + 'us-shareprices-daily.csv', delimiter=';')
    d["Date"]=pd.to_datetime(d["Date"])
    print('Stock Price data matrix is: ',d.shape)
    return d

In [13]:
def getYPricesReportDate(X, d, modifier=365):
    '''
    Get the stock prices for our X matrix to create Market Cap. column later.
    '''
    i=0
    y = [[None]*8 for i in range(len(X))] # Preallocation list of list of 2 [(price at date) (price at date + modifier)]
    whichDateCol='Publish Date'# or 'Report Date', is the performance date from->to. Want this to be publish date.
    # Because of time lag between report date (which can't be actioned on) and publish date (data we can trade with)
    for index in range(len(X)):
        y[i]=getYPriceDataNearDate(X['Ticker'].iloc[index], X[whichDateCol].iloc[index], 0, d)
        i=i+1
    return y

def getXFullDataMerged(myLocalPath='C:/Users/damon/OneDrive/BYO_Investing_AI/2024/Stock_Data/SimFin2024/'):
    '''
    For combining fundamentals financial data from SimFin+ only,
    without API. 
    Download Income Statement, Balance Sheet and Cash Flow files,
    the -full versions, e.g. us-balance-annual-full.csv.
    Place in a directory and give the directory path to the function.
    Assumes standard filenames from SimFin.
    Returns a DataFrame of the combined result. 
    Prints file infos.
    '''
    incomeStatementData=pd.read_csv(myLocalPath+'us-income-annual-full-asreported.csv',
                                    delimiter=';')
    balanceSheetData=pd.read_csv(myLocalPath+'us-balance-annual-full-asreported.csv',
                                 delimiter=';')
    CashflowData=pd.read_csv(myLocalPath+'us-cashflow-annual-full-asreported.csv',
                             delimiter=';')
    
    print('Income Statement CSV data is(rows, columns): ',
          incomeStatementData.shape)
    print('Balance Sheet CSV data is: ',
          balanceSheetData.shape)
    print('Cash Flow CSV data is: ' ,
          CashflowData.shape)
    

    # Merge the data together
    result = pd.merge(incomeStatementData, balanceSheetData,\
                on=['Ticker','SimFinId','Currency',
                    'Fiscal Year','Report Date','Publish Date'])
    
    result = pd.merge(result, CashflowData,\
                on=['Ticker','SimFinId','Currency',
                    'Fiscal Year','Report Date','Publish Date'])
    
    # dates in correct format
    result["Report Date"] = pd.to_datetime(result["Report Date"]) 
    result["Publish Date"] = pd.to_datetime(result["Publish Date"])
    
    print('Merged X data matrix shape is: ', result.shape)
    
    return result

In [14]:
X = getXFullDataMerged()

Income Statement CSV data is(rows, columns):  (53643, 68)
Balance Sheet CSV data is:  (53643, 95)
Cash Flow CSV data is:  (53643, 62)
Merged X data matrix shape is:  (40824, 213)


In [15]:
# Get data only for 2024
PublishDateStart = "2024-01-01"
PublishDateEnd = "2024-04-01"
bool_list = X['Publish Date'].between(\
              pd.to_datetime(PublishDateStart),\
              pd.to_datetime(PublishDateEnd) )
X=X[bool_list]

In [16]:
d=getYRawData2024()

Stock Price data matrix is:  (17999010, 11)


In [21]:
y = getYPricesReportDate(X, d) # Takes a min due to price lookups.
y = pd.DataFrame(y, columns=['Ticker', 'Open Price', 'Date', 'Volume'])

In [22]:
import yfinance as yf

# Replace 'AAPL' with the ticker symbol of the stock you want to fetch data for
ticker_symbol = 'WMT'

# Create a yfinance object for the specified stock
stock = yf.Ticker(ticker_symbol)

# Get historical data for the stock with the 'history' method
# By default, it will return data for the last available trading day
historical_data = stock.history(period="1d")

# Extract the open price from the historical data
open_price = historical_data['Open'].iloc[0]

print(f"Open price for {ticker_symbol}: {open_price}")


Open price for WMT: 61.31999969482422


In [23]:
import yfinance as yf
openPrices = {}
for sym in y['Ticker']:
    # Replace 'AAPL' with the ticker symbol of the stock you want to fetch data for
    ticker_symbol = sym

    # Create a yfinance object for the specified stock
    stock = yf.Ticker(ticker_symbol)
    # Get historical data for the stock with the 'history' method
    # By default, it will return data for the last available trading day
    historical_data = stock.history(period="1d")
    if not historical_data.empty:
        # Extract the open price from the historical data
        open_price = historical_data['Open'].iloc[0]

        print(f"Open price for {ticker_symbol}: {open_price}")
        openPrices[ticker_symbol] = open_price

Open price for AA: 31.0
Open price for AAL: 14.710000038146973
Open price for AAOI: 14.699999809265137
Open price for AAT: 21.469999313354492
Open price for AAWH: 1.2699999809265137
Open price for ABBV: 177.22999572753906
Open price for ABG: 224.55999755859375
Open price for ABIO: 1.7799999713897705
Open price for ABNB: 167.0
Open price for ABT: 111.5199966430664
Open price for ABUS: 2.549999952316284
Open price for ABVC: 1.0700000524520874
Open price for ACA: 85.66999816894531
Open price for ACAD: 18.34000015258789
Open price for ACCO: 5.71999979019165
Open price for ACEL: 12.010000228881836
Open price for ACHC: 75.22000122070312
Open price for ACHR: 5.090000152587891
Open price for ACIW: 32.689998626708984
Open price for ACLS: 113.4000015258789
Open price for ACLX: 72.54000091552734
Open price for ACMR: 30.190000534057617
Open price for ACRE: 7.809999942779541
Open price for ACRS: 1.2000000476837158
Open price for ACTG: 5.210000038146973
Open price for ACU: 42.810001373291016
Open pr

AMHCU: No data found, symbol may be delisted


Open price for AMK: 36.11000061035156
Open price for AMKR: 31.700000762939453
Open price for AMLX: 2.890000104904175
Open price for AMN: 66.70999908447266


AMOT: No data found, symbol may be delisted


Open price for AMPH: 44.45000076293945
Open price for AMPL: 11.520000457763672
Open price for AMPY: 6.489999771118164
Open price for AMRX: 6.289999961853027
Open price for AMWL: 0.9070000052452087
Open price for AN: 165.00999450683594
Open price for ANAB: 24.40999984741211
Open price for ANDE: 56.349998474121094
Open price for ANET: 303.0
Open price for ANGI: 2.7799999713897705
Open price for ANIK: 25.450000762939453
Open price for ANIP: 69.93000030517578
Open price for ANIX: 3.680000066757202
Open price for ANSS: 349.8299865722656
Open price for ANY: 1.3899999856948853
Open price for AOMR: 11.0
Open price for AON: 327.3999938964844
Open price for AOS: 89.54000091552734
Open price for APAM: 44.68000030517578
Open price for APEI: 14.239999771118164
Open price for APG: 39.18000030517578
Open price for APGE: 67.62999725341797
Open price for APH: 114.5
Open price for APLE: 16.799999237060547
Open price for APLS: 57.0
Open price for APPF: 233.89999389648438
Open price for APPN: 39.0
Open pr

...

Open price for WTS: 211.0500030517578
Open price for WTW: 274.8999938964844
Open price for WU: 13.819999694824219
Open price for WW: 1.9700000286102295
Open price for WWW: 10.420000076293945
Open price for WY: 35.59000015258789
Open price for WYNN: 100.62000274658203
Open price for X: 40.11000061035156
Open price for XBIT: 7.25
Open price for XEL: 52.5
Open price for XENE: 43.65999984741211
Open price for XFIN: 10.878000259399414
Open price for XMTR: 17.799999237060547
Open price for XNCR: 22.739999771118164
Open price for XOM: 113.4800033569336
Open price for XOMA: 24.290000915527344


XPDB: No price data found, symbol may be delisted (period=1d)


Open price for XPEL: 53.209999084472656
Open price for XPER: 11.220000267028809
Open price for XPO: 125.58000183105469
Open price for XPOF: 14.899999618530273
Open price for XPRO: 19.520000457763672
Open price for XRAY: 33.45000076293945
Open price for XRX: 17.200000762939453
Open price for XYL: 129.92999267578125
Open price for YELP: 38.83000183105469
Open price for YEXT: 6.230000019073486
Open price for YMAB: 16.979999542236328
Open price for YOU: 20.170000076293945
Open price for YUM: 136.61000061035156
Open price for YUMC: 38.459999084472656
Open price for Z: 50.9900016784668
Open price for ZBH: 127.2699966430664
Open price for ZETA: 11.15999984741211
Open price for ZEUS: 67.9800033569336
Open price for ZI: 16.020000457763672
Open price for ZIP: 12.079999923706055


ZNTE: No price data found, symbol may be delisted (period=1d)


Open price for ZM: 66.3499984741211
Open price for ZNTL: 13.84000015258789
Open price for ZTS: 168.77000427246094
Open price for ZUMZ: 14.609999656677246
Open price for ZUUS: 6.889999866485596


In [24]:
y2 = pd.DataFrame(list(openPrices.items()), columns=['Ticker', 'Open Price'])
y2['Date']='2024-03-22'
y2

,Ticker,Open Price,Date
0,AA,31.000000,2024-03-22
1,AAL,14.710000,2024-03-22
2,AAOI,14.700000,2024-03-22
3,AAT,21.469999,2024-03-22
4,AAWH,1.270000,2024-03-22
...,...,...,...
1589,ZM,66.349998,2024-03-22
1590,ZNTL,13.840000,2024-03-22
1591,ZTS,168.770004,2024-03-22
1592,ZUMZ,14.610000,2024-03-22


In [25]:
y2=y2.reset_index(drop=True)
X=X.reset_index(drop=True)

# Issue where no share price
bool_list1 = ~y["Open Price"].isnull()
# Issue where there is low/no volume
#bool_list2 = ~(y['Volume']<1e4)

y2=y2[bool_list1]
X=X[bool_list1]

# Issues where no listed number of shares
bool_list4 = ~X["Shares (Diluted)_x"].isnull()
y2=y2[bool_list4]
X=X[bool_list4]
               
y2=y2.reset_index(drop=True)
X=X.reset_index(drop=True)

X["Market Cap"] = y2["Open Price"]*X["Shares (Diluted)_x"]

C:\Users\damon\AppData\Local\Temp\ipykernel_11224\1517519545.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y2=y2[bool_list1]
C:\Users\damon\AppData\Local\Temp\ipykernel_11224\1517519545.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y2=y2[bool_list4]


In [26]:
X.to_csv("Annual_Stock_Price_Fundamentals_Filtered_2024_present.csv")
y2.to_csv("Tickers_Dates_2024_present.csv")

# process_x_y_learning_data

In [28]:
def fixNansInX(x):
    '''
    Takes in x DataFrame, edits it so that important keys
    are 0 instead of NaN.
    '''
    keyCheckNullList = ["Short Term Debt" ,\
            "Long Term Debt" ,\
            "Interest Expense, Net",\
            "Income Tax (Expense) Benefit, Net",\
            "Cash, Cash Equivalents & Short Term Investments",\
            "Property, Plant & Equipment, Net",\
            "Revenue",\
            "Gross Profit",\
            "Total Current Liabilities"]
    x[keyCheckNullList]=x[keyCheckNullList].fillna(0)
    x['Property, Plant & Equipment, Net'] = x['Property, Plant & Equipment, Net'].fillna(0)
    
def addColsToX(x):
    '''
    Takes in x DataFrame, edits it to include:
        Enterprise Value.
        Earnings before interest and tax.
    
    '''
    x["EV"] = x["Market Cap"] \
    + x["Long Term Debt"] \
    + x["Short Term Debt"] \
    - x["Cash, Cash Equivalents & Short Term Investments"]

    x["EBIT"] = x["Net Income (Common)"] \
    - x["Interest Expense, Net"] \
    - x["Income Tax (Expense) Benefit, Net"]

In [29]:
# Make new X with ratios to learn from.
def getXRatios(x_):
    '''
    Takes in x_, which is the fundamental stock DataFrame raw. 
    Outputs X, which is the data encoded into stock ratios.
    '''
    X=pd.DataFrame()
    
    # EV/EBIT
    X["EV/EBIT"] = x_["EV"] / x_["EBIT"]
    
    # Op. In./(NWC+FA)
    X["Op. In./(NWC+FA)"] = x_["Operating Income (Loss)"] \
    / (x_["Total Current Assets"] - x_["Total Current Liabilities"] \
       + x_["Property, Plant & Equipment, Net"])
    
    # P/E
    X["P/E"] = x_["Market Cap"] / x_["Net Income (Common)"]
    
    # P/B
    X["P/B"] = x_["Market Cap"] / x_["Total Equity"] 
    
    # P/S
    X["P/S"] = x_["Market Cap"] / x_["Revenue"] 
    
    # Op. In./Interest Expense
    X["Op. In./Interest Expense"] = x_["Operating Income (Loss)"]\
    / - x_["Interest Expense, Net"]
    
    # Working Capital Ratio
    X["Working Capital Ratio"] = x_["Total Current Assets"]\
    / x_["Total Current Liabilities"]
    
    # Return on Equity
    X["RoE"] = x_["Net Income (Common)"] / x_["Total Equity"]
    
    # Return on Capital Employed
    X["ROCE"] = x_["EBIT"]\
    / (x_["Total Assets"] - x_["Total Current Liabilities"] )
    
    # Debt/Equity
    X["Debt/Equity"] = x_["Total Liabilities"] / x_["Total Equity"]
    
    # Debt Ratio
    X["Debt Ratio"] = x_["Total Assets"] / x_["Total Liabilities"]
    
    # Cash Ratio
    X["Cash Ratio"] = x_["Cash, Cash Equivalents & Short Term Investments"]\
    / x_["Total Current Liabilities"]
    
    # Asset Turnover
    X["Asset Turnover"] = x_["Revenue"] / \
                            x_["Property, Plant & Equipment, Net"]
    
    # Gross Profit Margin
    X["Gross Profit Margin"] = x_["Gross Profit"] / x_["Revenue"]
    
    ### Altman ratios ###
    # (CA-CL)/TA
    X["(CA-CL)/TA"] = (x_["Total Current Assets"]\
                       - x_["Total Current Liabilities"])\
                        /x_["Total Assets"]
    
    # RE/TA
    X["RE/TA"] = x_["Retained Earnings"]/x_["Total Assets"]
    
    # EBIT/TA
    X["EBIT/TA"] = x_["EBIT"]/x_["Total Assets"]
    
    # Book Equity/TL
    X["Book Equity/TL"] = x_["Total Equity"]/x_["Total Liabilities"]
    
    X.fillna(0, inplace=True)
    return X

def fixXRatios(X):
    '''
    Takes in X, edits it to have the distributions clipped.
    The distribution clippings are done manually by eye,
    with human judgement based on the information.
    '''
    X["RoE"].clip(-5, 5, inplace=True)
    X["Op. In./(NWC+FA)"].clip(-5, 5, inplace=True)
    X["EV/EBIT"].clip(-500, 500, inplace=True)
    X["P/E"].clip(-1000, 1000, inplace=True)
    X["P/B"].clip(-50, 100, inplace=True)    
    X["P/S"].clip(0, 500, inplace=True)
    X["Op. In./Interest Expense"].clip(-600, 600, inplace=True)#-600, 600
    X["Working Capital Ratio"].clip(0, 30, inplace=True)  
    X["ROCE"].clip(-2, 2, inplace=True)
    X["Debt/Equity"].clip(0, 100, inplace=True)
    X["Debt Ratio"].clip(0, 50, inplace=True)  
    X["Cash Ratio"].clip(0, 30, inplace=True)
    X["Gross Profit Margin"].clip(0, 1, inplace=True) #how can be >100%?
    X["(CA-CL)/TA"].clip(-1.5, 2, inplace=True)
    X["RE/TA"].clip(-20, 2, inplace=True)
    X["EBIT/TA"].clip(-2, 1, inplace=True)
    X["Book Equity/TL"].clip(-2, 20, inplace=True)
    X["Asset Turnover"].clip(-2000, 2000, inplace=True)# 0, 500

In [30]:
X=pd.read_csv("Annual_Stock_Price_Fundamentals_Filtered_2024_present.csv", 
              index_col=0)

# Net Income fix, checked annual reports.
X['Net Income (Common)'] = X['Net Income_x'] 

fixNansInX(X)
addColsToX(X)
X=getXRatios(X)
fixXRatios(X)
X.to_csv("Annual_Stock_Price_Fundamentals_Ratios_2024.csv")

# final_stock_selection

In [40]:
def calcZScores(X):
    '''
    Calculate Altman Z'' scores 1995
    '''
    Z = pd.DataFrame()
    Z['Z score'] = 3.25 \
    + 6.51 * X['(CA-CL)/TA']\
    + 3.26 * X['RE/TA']\
    + 6.72 * X['EBIT/TA']\
    + 1.05 * X['Book Equity/TL']
    return Z

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import pickle

In [37]:
# Linear model pipeline
def trainLinearModel(X_train, y_train):
    pl_linear = Pipeline([('Power Transformer', PowerTransformer()),
        ('linear', LinearRegression())])
    pl_linear.fit(X_train, y_train)
    return pl_linear

# ElasticNet model pipeline
def trainElasticNetModel(X_train, y_train):
    pl_ElasticNet = Pipeline([('Power Transformer', PowerTransformer()),
        ('ElasticNet', ElasticNet(l1_ratio=0.00001))])
    pl_ElasticNet.fit(X_train, y_train)
    return pl_ElasticNet

# KNeighbors regressor
def trainKNeighborsModel(X_train, y_train):
    pl_KNeighbors = Pipeline([('Power Transformer', PowerTransformer()),
        ('KNeighborsRegressor', KNeighborsRegressor(n_neighbors=40))])
    pl_KNeighbors.fit(X_train, y_train)
    return pl_KNeighbors

# DecisionTreeRegressor
def traindecTreeModel(X_train, y_train):
    pl_decTree = Pipeline([
        ('DecisionTreeRegressor',\
         DecisionTreeRegressor(max_depth=20, random_state=42))
    ])
    pl_decTree.fit(X_train, y_train)
    return pl_decTree

# RandomForestRegressor
def trainrfregressorModel(X_train, y_train):
    pl_rfregressor = Pipeline([
        ('RandomForestRegressor',\
         RandomForestRegressor(max_depth=10, random_state=42))
    ])
    pl_rfregressor.fit(X_train, y_train)
    
    return pl_rfregressor

# GradientBoostingRegressor
def traingbregressorModel(X_train, y_train):
    pl_GradBregressor = Pipeline([
        ('GradBoostRegressor',\
         GradientBoostingRegressor(n_estimators=100,\
                                   learning_rate=0.1,\
                                   max_depth=14,\
                                   random_state=42,\
                                   loss='absolute_error'))  ])
    pl_GradBregressor.fit(X_train, y_train)
    
    return pl_GradBregressor

# SVM
def trainsvmModel(X_train, y_train):
    pl_svm = Pipeline([('Power Transformer', PowerTransformer()),
        ('SVR', SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1))])
    pl_svm.fit(X_train, y_train)
    return pl_svm

In [48]:
import pandas as pd

def pickStockForMe():
    '''
    Pick stocks.
    Reads Annual_Stock_Price_Fundamentals_Ratios.csv,
    and Annual_Stock_Price_Performance_Percentage.csv,
    trains the AI with the best model/parameters,
    Then picks stocks using outputs from Notebooks 1 and 2:
    Annual_Stock_Price_Fundamentals_Ratios_2021.csv,
    and Tickers_Dates_2021.csv.
    Outputs a DataFrame of best picks.
    '''
    # Training X and Y of all previous year data
    X=pd.read_csv("Annual_Stock_Price_Fundamentals_Ratios.csv", 
                  index_col=0)
    # annual stock performances
    yperf=pd.read_csv("Annual_Stock_Price_Performance_Percentage.csv", 
                      index_col=0)
    
    yperf=yperf["Perf"]

    # Stock selection ratios for 2024 X
    X_2024=pd.read_csv("Annual_Stock_Price_Fundamentals_Ratios_2024.csv"
                       , index_col=0)
    # And the row tickers
    tickers=pd.read_csv("Tickers_Dates_2024_present.csv", index_col=0)
    
    # Gradient Boosted tree
    #model_pl = traingbregressorModel(X, yperf)
    #y_pred=model_pl.predict(X_2024)
    #y_pred=pd.DataFrame(y_pred)
    
    # Random Forest
    #model_pl = trainrfregressorModel(X, yperf)
    #y_pred=model_pl.predict(X_2024)
    #y_pred=pd.DataFrame(y_pred)
    
    
    # KNN
    model_pl = trainKNeighborsModel(X, yperf)
    y_pred=model_pl.predict(X_2024)
    y_pred=pd.DataFrame(y_pred)
    
    

    # FINAL STOCK PICKS
    # Separate out stocks with low Z scores
    # 3.75 is approx. B- rating
    z = calcZScores(X_2024)
    zbl = (z['Z score'].reset_index(drop=True) > 3) 

    # Final_Predictions = pd.DataFrame()
    Final_Predictions=tickers[['Ticker','Date']].reset_index(drop=True)\
                                                [zbl].reset_index(drop=True)
    
    Final_Predictions['Perf'] = y_pred.reset_index(drop=True)\
                                                [zbl].reset_index(drop=True)

    return Final_Predictions.sort_values(by='Perf', ascending=False)\
                                                .reset_index(drop=True)

In [49]:
# What it's all about.
res = pickStockForMe()

C:\Users\damon\AppData\Local\Temp\ipykernel_11224\2136070793.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Final_Predictions=tickers[['Ticker','Date']].reset_index(drop=True)\


In [43]:
# RF
pd.set_option('display.max_rows', 10000)
res

,Ticker,Date,Perf
0,MHK,2024-03-22,7.245068
1,CLSD,2024-03-22,4.372626
2,NCSM,2024-03-22,4.174444
3,NAUT,2024-03-22,3.423917
4,NERV,2024-03-22,3.131116
5,HOUS,2024-03-22,3.073291
6,HLF,2024-03-22,2.986561
7,STEM,2024-03-22,2.887061
8,NEPH,2024-03-22,2.717309
9,VSEC,2024-03-22,2.714641


In [50]:
# KNN (apparent 2nd best)
pd.set_option('display.max_rows', 10000)
res

,Ticker,Date,Perf
0,FUST,2024-03-22,1.848834
1,CRMD,2024-03-22,1.224183
2,FPH,2024-03-22,1.114430
3,CPSS,2024-03-22,1.111173
4,FTK,2024-03-22,1.106267
5,CTV,2024-03-22,1.072701
6,IVDN,2024-03-22,1.004654
7,MRTN,2024-03-22,1.002736
8,CMPS,2024-03-22,0.960102
9,FTCI,2024-03-22,0.955339


In [47]:
# GBST (apparent best)
pd.set_option('display.max_rows', 10000)
res

,Ticker,Date,Perf
0,FOA,2024-03-22,2.843751
1,KRMD,2024-03-22,2.508249
2,BNGO,2024-03-22,2.342310
3,QRTEA,2024-03-22,2.256590
4,NINE,2024-03-22,2.064774
5,HRTX,2024-03-22,1.873682
6,DMTK,2024-03-22,1.801671
7,NCSM,2024-03-22,1.645775
8,KNSA,2024-03-22,1.593723
9,DOUG,2024-03-22,1.533013
